# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [4]:
# Write your code below.
%load_ext dotenv 
%dotenv 

In [6]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [ ]:
import os
from glob import glob

# Write your code below.
# Load all parquet files from PRICE_DATA environment variable path
PRICE_DATA = os.getenv("PRICE_DATA")
parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive = True)

#print(parquet_files)
dd_px = dd.read_parquet(parquet_files).set_index("ticker")
print(dd_px.compute().head())

             Date       Open       High        Low      Close  Adj Close  \
ticker                                                                     
BBUS   2019-03-13  50.630001  50.669998  50.513000  50.513000  49.567730   
BBUS   2019-03-14  50.500000  50.542000  50.439999  50.500000  49.554974   
BBUS   2019-03-15  50.599998  50.698002  50.599998  50.698002  49.749268   
BBUS   2019-03-18  50.860001  50.946999  50.750000  50.910999  49.958282   
BBUS   2019-03-19  51.150002  51.169998  50.855999  50.863998  49.912159   

         Volume    source  Year  
ticker                           
BBUS    27300.0  BBUS.csv  2019  
BBUS     4000.0  BBUS.csv  2019  
BBUS      900.0  BBUS.csv  2019  
BBUS    16900.0  BBUS.csv  2019  
BBUS    14800.0  BBUS.csv  2019  


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [16]:
# Write your code below.

#Add lagged features and calculate returns and hi_lo_range
dd_cls = dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1))
)

dd_adj= dd_cls.groupby('ticker', group_keys=False).apply(
lambda x: x.assign(Adj_Close_lag_1 = x['Adj Close'].shift(1))
)

dd_rets = dd_adj.assign(
    returns = lambda x: x['Close']/x['Close_lag_1'] - 1,
    hi_lo_range = lambda x: x['High'] - x['Low']
)

df_feat = dd_rets.compute()
df_feat

C:\Users\sunib\AppData\Local\Temp\ipykernel_25908\148348221.py:4: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_cls = dd_px.groupby('ticker', group_keys=False).apply(
C:\Users\sunib\AppData\Local\Temp\ipykernel_25908\148348221.py:8: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_adj= dd_cls.groupby('ticker', group_keys=False).apply(


,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range
ticker,,,,,,,,,,,,,
BBUS,2019-03-13,50.630001,50.669998,50.513000,50.513000,49.567730,27300.0,BBUS.csv,2019,NaN,NaN,NaN,0.156998
BBUS,2019-03-14,50.500000,50.542000,50.439999,50.500000,49.554974,4000.0,BBUS.csv,2019,50.513000,49.567730,-0.000257,0.102001
BBUS,2019-03-15,50.599998,50.698002,50.599998,50.698002,49.749268,900.0,BBUS.csv,2019,50.500000,49.554974,0.003921,0.098003
BBUS,2019-03-18,50.860001,50.946999,50.750000,50.910999,49.958282,16900.0,BBUS.csv,2019,50.698002,49.749268,0.004201,0.196999
BBUS,2019-03-19,51.150002,51.169998,50.855999,50.863998,49.912159,14800.0,BBUS.csv,2019,50.910999,49.958282,-0.000923,0.313999
...,...,...,...,...,...,...,...,...,...,...,...,...,...
XMHQ,2020-03-26,41.830002,41.830002,41.830002,41.830002,41.830002,100.0,XMHQ.csv,2020,39.830002,39.830002,0.050213,0.000000
XMHQ,2020-03-27,40.299999,40.549999,40.110001,40.480000,40.480000,1900.0,XMHQ.csv,2020,41.830002,41.830002,-0.032274,0.439999
XMHQ,2020-03-30,40.810001,41.970001,40.810001,41.970001,41.970001,2000.0,XMHQ.csv,2020,40.480000,40.480000,0.036808,1.160000


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [17]:
import pandas as pd
# Verify the type of df_feat - Pandas DataFrame
print(type(df_feat))

<class 'pandas.core.frame.DataFrame'>


In [ ]:
# Write your code below.

# Convert the "Date" column to datetime format
df_feat["Date"] = pd.to_datetime(df_feat["Date"])

#Sort the DataFrame by ticker and Date
df_feat = df_feat.sort_values(["ticker", "Date"])
 
# Calculate the 10 day moving average of the "returns" column for each ticker
df_feat['returns_ma10'] = df_feat.groupby('ticker')['returns'].transform(lambda x: x.rolling(10).mean())

print(df_feat.head(10))


             Date       Open       High        Low      Close  Adj Close  \
ticker                                                                     
BBUS   2019-03-13  50.630001  50.669998  50.513000  50.513000  49.567730   
BBUS   2019-03-14  50.500000  50.542000  50.439999  50.500000  49.554974   
BBUS   2019-03-15  50.599998  50.698002  50.599998  50.698002  49.749268   
BBUS   2019-03-18  50.860001  50.946999  50.750000  50.910999  49.958282   
BBUS   2019-03-19  51.150002  51.169998  50.855999  50.863998  49.912159   
BBUS   2019-03-20  50.900002  50.950001  50.590000  50.779999  49.829735   
BBUS   2019-03-21  50.599998  51.400002  50.599998  51.334000  50.373367   
BBUS   2019-03-22  51.169998  51.180000  50.330002  50.360001  49.417591   
BBUS   2019-03-25  50.200001  50.400002  50.070000  50.307999  49.366562   
BBUS   2019-03-26  50.660000  50.779999  50.436001  50.632000  49.684502   

         Volume    source  Year  Close_lag_1  Adj_Close_lag_1   returns  \
ticker      

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

It wasn’t strictly necessary to convert the Dask DataFrame to pandas to calculate the moving average, because Dask can do rolling calculations too. However, using Pandas is much easier and more reliable, especially when working with groups like different tickers. Dask is better for very large datasets, but its rolling functions are more limited and harder to use. If data is small enough to fit in memory, pandas is simpler and easier.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.